# Homework - Grover MaxCut

The places where you have enter code are marked with `# YOUR CODE HERE`.

In [56]:
import cirq
from cirq import H, X, Y, Z, CX, inverse

## Question 1 (4 points)

Write a function, `oracle010`, that implements an oracle that marks the state $|010 \rangle$. The function `oracle010` has

* input: `qq`, a 3-qubit register 
* returns: `None`

The function should append a sequence of gates to `qq` to mark the state $|010\rangle$ only. Don't append any measurements to `qq`.

To help you test the function, we have provided the `grover_diffusion` and `grover` functions.

In [4]:
def oracle010(qq):
    yield X(qq[0])
    yield X(qq[2])
    yield Z(qq[1]).controlled_by(qq[0],qq[2])
    yield X(qq[0])
    yield X(qq[2])



In [5]:
# visualize your implemented gates
qqTest = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(oracle010(qqTest))
circuit

0: ───X───@───X───
          │
1: ───────@───────
          │
2: ───X───@───X───

In [14]:
# To check your solution, we need some to implement grover
def grover_diffusion(qq,n):
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[n-1]).controlled_by(*(qq[0:n-1]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

In [15]:
def grover(trials_number):
    n=3
    qq = cirq.LineQubit.range(n)
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*qq))  

    for i in range(2):
        circuit.append(oracle010(qq))
        circuit.append(grover_diffusion(qq,n))
    circuit.append(cirq.measure(*qq, key='result'))

    # determine the statistics of the measurements
    s = cirq.Simulator() 
    samples = s.run(circuit, repetitions=trials_number)

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    print(counts)
    return counts.get('010')

In [16]:
# run grover to test if your function gives the right answer
grover(100)

Counter({'010': 94, '110': 3, '000': 3})


94

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 2 (6 points)

Graph $G$ has 5 vertices and 6 edges: (0,3), (0,4), (1,3), (1,4), (2,3), (2,4).

Write an oracle for the graph $G$ to check whether it admits a valid 2-coloring. 

The function `oracle2` has

* input: `qq`, a 12-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 0-4 for the vertices, 5-10 for the edges and 11 as the ancilla. 

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation2` functions.

In [57]:
def oracle2(qq):

    def edge_check(a, b, c):
        yield cirq.CNOT(qq[a], qq[c])
        yield cirq.CNOT(qq[b], qq[c])

    yield edge_check(0, 3, 5)
    yield edge_check(0, 4, 6)
    yield edge_check(1, 3, 7)
    yield edge_check(1, 4, 8)
    yield edge_check(2, 3, 9)
    yield edge_check(2, 4, 10)

    yield cirq.X(qq[11]).controlled_by(*qq[5:11])



In [58]:
# We need some code so you can check your solution
def oracle_computation2(qq):
    yield oracle2(qq)
    yield Z(qq[11])
    yield inverse(oracle2(qq))  

In [59]:
def grover2(trials_number):    
    import cirq
    from cirq import X, H, Z, inverse, CX
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(12)
    n=5
    
    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation2(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [60]:
#You can use this cell to test your solution
shots=1000
grover2(shots)

Counter({'11100': 467,
         '00011': 433,
         '10100': 9,
         '10000': 7,
         '01110': 6,
         '11010': 6,
         '11001': 6,
         '00010': 5,
         '01010': 5,
         '11101': 5,
         '00100': 4,
         '11110': 4,
         '11111': 4,
         '00101': 4,
         '00000': 4,
         '11011': 3,
         '00111': 3,
         '10101': 3,
         '11000': 2,
         '10111': 2,
         '00001': 2,
         '10010': 2,
         '01100': 2,
         '01101': 2,
         '01011': 2,
         '10011': 2,
         '01111': 2,
         '10001': 2,
         '00110': 1,
         '01000': 1})

In [ ]:
# hidden tests in this cell will be used for grading.

## Question 3 (10 points)

Graph $G$ has 4 vertices and 5 edges: (0,1), (0,2), (0,3), (1,2), (1,3)

Write an oracle for the graph $G$ to check whether there exists a coloring with at least 4 edges connecting vertices with different colors.

The function `oracle3` has

* input: `qq`, a 13-qubit register 
* returns: `None`

The function should append only a sequence of gates to `qq`. It should not append any measurements to `qq`.

Use qubits 
- 0-3 for the vertices,
- 4-8 for the edges,
- 9-11 for the addition (remember we need three qubits here for addition unlike the last question), and
- 12 as the ancilla.

You can test the oracle with the provided `grover_diffusion`, `grover` and `oracle_computation3` functions.

In [103]:
def oracle3(qq):
    def edge_check(a, b, c):
        yield CX(qq[a], qq[c])
        yield CX(qq[b], qq[c])

    yield edge_check(0, 1, 4)
    yield edge_check(0, 2, 5)
    yield edge_check(0, 3, 6)
    yield edge_check(1, 2, 7)
    yield edge_check(1, 3, 8)

    # Edges checking
    yield CX(qq[4],qq[9])

    # add qubits 5,7
    for i in range(5,8):
        yield CCX(qq[i],qq[9],qq[10]) 
        yield CX(qq[i],qq[9])
        
    # add qubits 8
    yield X(qq[11]).controlled_by(qq[8],qq[9],qq[10])
    yield CCX(qq[8],qq[9],qq[10]) 
    yield CX(qq[8],qq[9])

    yield X(qq[9])
    yield X(qq[10])
    yield X(qq[12]).controlled_by(*(qq[8:11]))

In [104]:
# We need some code so you can check your solution
def oracle_computation3(qq):
    yield oracle3(qq)
    yield Z(qq[12])
    yield inverse(oracle3(qq))  

In [105]:
import cirq
from cirq import X, H, Z, inverse, CX, CCX
    
def grover3(trials_number):    
    s = cirq.Simulator()

    qq = cirq.LineQubit.range(13)
    n=4

    circuit = cirq.Circuit()
    circuit.append(H.on_each(*(qq[0:n])))
    for i in range(2):
        circuit.append(oracle_computation3(qq))
        circuit.append(grover_diffusion(qq,n))

    circuit.append(cirq.measure(*(qq[0:n]), key='result'))

    # determine the statistics of the measurements
    samples = s.run(circuit, repetitions=trials_number)
    result = samples.measurements["result"]

    def bitstring(bits):
        return "".join(str(int(b)) for b in bits)

    counts = samples.histogram(key="result",fold_func=bitstring)
    return counts

In [106]:
#You can use this cell to test your solution
shots=1000
grover3(shots)

Counter({'1100': 473,
         '0011': 466,
         '0110': 9,
         '1101': 7,
         '1010': 6,
         '1011': 6,
         '0001': 5,
         '0101': 5,
         '0010': 4,
         '0000': 4,
         '1111': 4,
         '1110': 3,
         '0100': 3,
         '0111': 3,
         '1001': 1,
         '1000': 1})

In [ ]:
# hidden tests in this cell will be used for grading.